In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
url='https://www1.folha.uol.com.br/poder/'

#extraindo o link da manchete principal
pg=requests.get(url)
html=BeautifulSoup(pg.content, 'html.parser')
main_headline=html.find('a', attrs={'class': 'c-main-headline__url'})
url_manchete=main_headline['href']                                   #=============== VAR BANCO DE DADOS

#extraindo os demais links
urls = html.find_all('a', {'class': 'c-headline__url'})
manchetes=[]
for link in urls:
    if link['href']!='https://www1.folha.uol.com.br/especial/2019/cafe-da-manha/':
        manchetes.append(link['href'])
manchetes.append(url_manchete)

#acessando o link extraído da pagina principal
for url in manchetes:
    pg=requests.get(url)
    html=BeautifulSoup(pg.content, 'html5lib')

    #retirando título e data
    titulo=re.sub("[0-9]{2}[/][0-9]{2}[/][0-9]{4}", "", html.find('title').text)
    titulo=re.sub(" - Poder - Folha", "", titulo)
    titulo=re.sub("-", "", titulo.strip())
    data=re.search("[0-9]{2}[/][0-9]{2}[/][0-9]{4}", html.find('title').text).group()

    #retirando conteudo
    div=html.find('div', attrs={'class': 'c-news__body'})
    paragrafos=div.findChildren("p" , recursive=False)
    conteudo=""
    for p in paragrafos:
        conteudo+=p.text
    
    arquivo_bd=('reportagens_folha.csv')
    bd=pd.read_csv(arquivo_bd)
    
    if (url in list(bd['url']))==False:
        novo=pd.DataFrame(data={'data':str(data), 'titulo':str(titulo), 'url': str(url), 'conteudo':str(conteudo)}, index=[0])
        bd=bd.append(novo, ignore_index=True)
        #salvando de volta o csv atualizado
        print("Salvando nova manchete %s"  % url)
        bd.to_csv(arquivo_bd, index=False)
    else:
        print("Manchete já salva: %s" % url)


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Salvando nova manchete https://www1.folha.uol.com.br/poder/2020/09/atuacao-de-covas-na-pandemia-e-aprovada-por-37-e-reprovada-por-24-segundo-datafolha.shtml
Salvando nova manchete https://www1.folha.uol.com.br/poder/2020/09/impeachment-e-puramente-politico-e-terceiro-turno-da-eleicao-diz-governador-de-sc.shtml
Salvando nova manchete https://www1.folha.uol.com.br/poder/2020/09/bolsonaro-diz-que-pode-entrar-em-eleicao-para-influenciar-em-sao-paulo-santos-e-manaus.shtml
Salvando nova manchete https://www1.folha.uol.com.br/poder/2020/09/ao-menos-21-mil-candidatos-mudaram-declaracao-de-cor-para-eleicao-de-2020.shtml


In [3]:
    #salvando no csv
    #primeiro lê o csv já gravado para identificar se a manchete atual já foi gravada

